In [4]:
# importing necessary libraries for data loading and EDA
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, auc, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')

In [5]:
# Loading the train data
df_train = pd.read_csv(r'/home/ducanh/Credit Card Transactions Fraud Detection/Datasets/fraudTrain.csv')
df_test = pd.read_csv(r'/home/ducanh/Credit Card Transactions Fraud Detection/Datasets/fraudTest.csv')
df = pd.concat([df_train, df_test])

In [6]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['trans_hour'] = df['trans_date_trans_time'].dt.time.apply(lambda x: str(x)[:2])

df['dob'] = pd.to_datetime(df['dob'])
df['cust_age'] = df['dob'].dt.year.apply(lambda x: 2021-x)
df['cust_age_groups'] = df['cust_age'].apply(lambda x: 'below 10' if x<10 else ('10-20' if x>=10 and x<20 else ('20-30' if x>=20 and x<30 else('30-40' if x>=30 and x<40 else('40-50' if x>=40 and x<50 else('50-60' if x>=50 and x<60 else('60-70' if x>=60 and x<70 else ('70-80' if x>=70 and x<80 else 'Above 80'))))))))

In [7]:
drop_col = ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant','first', 'last', 'street', 'city', 'state', 'lat',
       'long','dob', 'unix_time', 'cust_age', 'merch_lat',
       'merch_long', 'city_pop']
df.drop(drop_col, axis = 1, inplace = True)

In [8]:
age_piv_2 = pd.pivot_table(data = df,
                           index = 'cust_age_groups',
                           columns = 'is_fraud',
                           values = 'amt',
                           aggfunc = np.mean)
age_piv_2.sort_values(by = 1, ascending = True, inplace = True)
age_dic = {k:v for (k,v) in zip(age_piv_2.index.values, age_piv_2.reset_index().index.values)}
df['cust_age_groups'] = df['cust_age_groups'].map(age_dic)

In [9]:
merch_cat = df[df['is_fraud'] == 1].groupby('category')['amt'].mean().sort_values(ascending = True)
merch_cat_dic = {k:v for (k,v) in zip(merch_cat.index.values,merch_cat.reset_index().index.values)}
df['category'] = df['category'].map(merch_cat_dic)

In [10]:
job_txn_piv_2 = pd.pivot_table(data = df,
                               index = 'job',
                               columns = 'is_fraud',
                               values= 'amt',
                               aggfunc = np.mean)
job_cat_dic = {k:v for (k,v) in zip(job_txn_piv_2.index.values, job_txn_piv_2.reset_index().index.values)}
df['job'] = df['job'].map(job_cat_dic)

In [11]:
df['trans_hour'] = df['trans_hour'].astype('int')
df = pd.get_dummies(data  = df, columns = ['gender'], drop_first = True, dtype = 'int')

# Train_test_split

In [12]:
train,test = train_test_split(df, test_size=0.33, random_state=42, stratify = df['is_fraud'])
# visualizing class imbalance
df['is_fraud'].value_counts()
# check
print(train.shape)
print(test.shape)

# let's drop transaction number columns from both the training and testing data
train.drop('trans_num',axis = 1, inplace = True)
test.drop('trans_num',axis = 1, inplace = True)
# splitting data into dependent and independent features respectively
y_train = train['is_fraud']
X_train = train.drop('is_fraud',axis = 1)

y_test = test['is_fraud']
X_test = test.drop('is_fraud',axis = 1)

print('Shape of training data: ',(X_train.shape,y_train.shape))
print('Shape of testing data: ',(X_test.shape,y_test.shape))

(1241103, 9)
(611291, 9)
Shape of training data:  ((1241103, 7), (1241103,))
Shape of testing data:  ((611291, 7), (611291,))


In [13]:
sc = StandardScaler()
# scaling the training and testing data
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)
# convert them into dataframes
X_train_sc = pd.DataFrame(data = X_train_sc, columns = X_train.columns)
X_test_sc = pd.DataFrame(data = X_test_sc, columns = X_test.columns)

# XGBoost

In [15]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]), random_state=42, n_estimators=100)
xgb_model.fit(X_train_sc, y_train)

y_pred_train_proba = xgb_model.predict_proba(X_train_sc)
y_pred_test_proba = xgb_model.predict_proba(X_test_sc)

y_train_results = pd.DataFrame(y_pred_train_proba, columns=['pred_not_fraud', 'pred_fraud'])
y_test_results = pd.DataFrame(y_pred_test_proba, columns=['pred_not_fraud', 'pred_fraud'])

y_train_results['y_train_actual'] = y_train.values
y_test_results['y_test_actual'] = y_test.values

In [16]:
numbers = [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in numbers:
    y_train_results[i] = y_train_results.pred_fraud.map(lambda x: 1 if x > i else 0)
    y_test_results[i] = y_test_results.pred_fraud.map(lambda x: 1 if x > i else 0)

cutoff_df = pd.DataFrame(columns=['Threshold', 'precision_score', 'recall_score', 'F1_score', 'Accuracy'])

In [17]:
for i in numbers:
    cm1 = confusion_matrix(y_train_results['y_train_actual'], y_train_results[i])
    TP, FP, FN, TN = cm1[1,1], cm1[0,1], cm1[1,0], cm1[0,0]
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score_value = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    cutoff_df.loc[i] = [i, precision, recall, f1_score_value, accuracy]

print(cutoff_df)

best_idx = cutoff_df['F1_score'].idxmax()
best_threshold = cutoff_df.loc[best_idx, 'Threshold']
best_f1_score = cutoff_df.loc[best_idx, 'F1_score']
best_precision = cutoff_df.loc[best_idx, 'precision_score']
best_recall = cutoff_df.loc[best_idx, 'recall_score']
best_accuracy = cutoff_df.loc[best_idx, 'Accuracy']
best_auc = roc_auc_score(y_train_results['y_train_actual'], y_train_results[best_threshold])

print(f'Best Threshold: {best_threshold:.4f}')
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')
print(f'Best F1 Score: {best_f1_score:.4f}')
print(f'Best ROC_AUC Score: {best_auc:.4f}')

      Threshold  precision_score  recall_score  F1_score  Accuracy
0.10       0.10         0.157792      1.000000  0.272574  0.972192
0.15       0.15         0.189252      1.000000  0.318271  0.977681
0.20       0.20         0.218151      1.000000  0.358168  0.981328
0.30       0.30         0.268901      1.000000  0.423833  0.985835
0.40       0.40         0.318036      1.000000  0.482591  0.988828
0.50       0.50         0.367157      1.000000  0.537110  0.991020
0.60       0.60         0.418746      0.999072  0.590143  0.992770
0.70       0.70         0.478022      0.997371  0.646290  0.994312
0.80       0.80         0.549516      0.991185  0.707044  0.995721
0.90       0.90         0.658277      0.966749  0.783235  0.997212
Best Threshold: 0.9000
Best Accuracy: 0.9972
Best Precision: 0.6583
Best Recall: 0.9667
Best F1 Score: 0.7832
Best ROC_AUC Score: 0.9821


In [18]:
for i in numbers:
    cm1 = confusion_matrix(y_test_results['y_test_actual'], y_test_results[i])
    TP, FP, FN, TN = cm1[1,1], cm1[0,1], cm1[1,0], cm1[0,0]
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score_value = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    cutoff_df.loc[i] = [i, precision, recall, f1_score_value, accuracy]

print(cutoff_df)

best_idx = cutoff_df['F1_score'].idxmax()
best_threshold = cutoff_df.loc[best_idx, 'Threshold']
best_f1_score = cutoff_df.loc[best_idx, 'F1_score']
best_precision = cutoff_df.loc[best_idx, 'precision_score']
best_recall = cutoff_df.loc[best_idx, 'recall_score']
best_accuracy = cutoff_df.loc[best_idx, 'Accuracy']
best_auc = roc_auc_score(y_test_results['y_test_actual'], y_test_results[best_threshold])

print(f'Best Threshold: {best_threshold:.4f}')
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')
print(f'Best F1 Score: {best_f1_score:.4f}')
print(f'Best ROC_AUC Score: {best_auc:.4f}')

      Threshold  precision_score  recall_score  F1_score  Accuracy
0.10       0.10         0.156484      0.984301  0.270037  0.972273
0.15       0.15         0.185068      0.976766  0.311178  0.977469
0.20       0.20         0.211867      0.974254  0.348046  0.980983
0.30       0.30         0.258225      0.963579  0.407299  0.985388
0.40       0.40         0.302316      0.958870  0.459697  0.988256
0.50       0.50         0.346057      0.953532  0.507817  0.990370
0.60       0.60         0.391197      0.945997  0.553504  0.992048
0.70       0.70         0.447748      0.936264  0.605790  0.993651
0.80       0.80         0.513528      0.917739  0.658556  0.995042
0.90       0.90         0.620223      0.891680  0.731582  0.996591
Best Threshold: 0.9000
Best Accuracy: 0.9966
Best Precision: 0.6202
Best Recall: 0.8917
Best F1 Score: 0.7316
Best ROC_AUC Score: 0.9444
